In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile10 = pd.read_pickle("BBB_Train_Decile10.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile10.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile10, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 38/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3127 - acc: 0.8895 - val_loss: 0.2927 - val_acc: 0.8912
Epoch 39/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3106 - acc: 0.8889 - val_loss: 0.2907 - val_acc: 0.8938
Epoch 40/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3083 - acc: 0.8931 - val_loss: 0.2956 - val_acc: 0.8873
Epoch 41/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3079 - acc: 0.8869 - val_loss: 0.2937 - val_acc: 0.8899
Epoch 42/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3018 - acc: 0.8895 - val_loss: 0.2930 - val_acc: 0.8886
Epoch 43/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3087 - acc: 0.8905 - val_loss: 0.2926 - val_acc: 0.8899
Epoch 44/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3044 - acc: 0.8892 - val_loss: 0.2883 - val_acc: 0.8925
Epoch 45/100
3087/3087 [=================

3087/3087 [==============================] - 14s 4ms/step - loss: 0.2778 - acc: 0.8963 - val_loss: 0.2690 - val_acc: 0.9003
Epoch 55/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2761 - acc: 0.8928 - val_loss: 0.2690 - val_acc: 0.9003
Epoch 56/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2766 - acc: 0.8950 - val_loss: 0.2689 - val_acc: 0.9003
Epoch 57/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2768 - acc: 0.8963 - val_loss: 0.2689 - val_acc: 0.9003
Epoch 58/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2805 - acc: 0.8941 - val_loss: 0.2690 - val_acc: 0.9003
Epoch 59/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2752 - acc: 0.8957 - val_loss: 0.2689 - val_acc: 0.9003
Epoch 60/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2734 - acc: 0.8944 - val_loss: 0.2689 - val_acc: 0.9003
Epoch 61/150
3087/3087 [==============================

Epoch 114/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2698 - acc: 0.8993 - val_loss: 0.2687 - val_acc: 0.9003
Epoch 115/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2699 - acc: 0.8983 - val_loss: 0.2687 - val_acc: 0.9003
Epoch 116/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2683 - acc: 0.8950 - val_loss: 0.2687 - val_acc: 0.9003
Epoch 117/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2748 - acc: 0.8970 - val_loss: 0.2687 - val_acc: 0.9003
Epoch 118/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2718 - acc: 0.8960 - val_loss: 0.2688 - val_acc: 0.9003
Epoch 119/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2717 - acc: 0.8918 - val_loss: 0.2688 - val_acc: 0.9003
Epoch 120/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2711 - acc: 0.8980 - val_loss: 0.2688 - val_acc: 0.9003
Epoch 121/150
3087/3087 [=========

3087/3087 [==============================] - 16s 5ms/step - loss: 0.2851 - acc: 0.8931 - val_loss: 0.2759 - val_acc: 0.9028
Epoch 82/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2884 - acc: 0.8931 - val_loss: 0.2758 - val_acc: 0.9028
Epoch 83/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2835 - acc: 0.8895 - val_loss: 0.2758 - val_acc: 0.9028
Epoch 84/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2884 - acc: 0.8905 - val_loss: 0.2757 - val_acc: 0.9028
Epoch 85/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2861 - acc: 0.8925 - val_loss: 0.2758 - val_acc: 0.9028
Epoch 86/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2871 - acc: 0.8921 - val_loss: 0.2758 - val_acc: 0.9028
Epoch 87/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2865 - acc: 0.8915 - val_loss: 0.2758 - val_acc: 0.9028
Epoch 88/150
3087/3087 [==============================

...

3087/3087 [==============================] - 9s 3ms/step - loss: 0.2713 - acc: 0.8989 - val_loss: 0.2695 - val_acc: 0.8990
Epoch 142/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2655 - acc: 0.8996 - val_loss: 0.2695 - val_acc: 0.9003
Epoch 143/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2670 - acc: 0.8996 - val_loss: 0.2694 - val_acc: 0.9003
Epoch 144/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2659 - acc: 0.8976 - val_loss: 0.2694 - val_acc: 0.8990
Epoch 145/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2649 - acc: 0.8983 - val_loss: 0.2694 - val_acc: 0.9003
Epoch 146/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2685 - acc: 0.9009 - val_loss: 0.2694 - val_acc: 0.8990
Epoch 147/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2677 - acc: 0.8983 - val_loss: 0.2694 - val_acc: 0.8990
Epoch 148/200
3087/3087 [==============================

Test accuracy: 0.9028497409326425
Train on 3087 samples, validate on 772 samples
Epoch 1/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.6660 - acc: 0.5653 - val_loss: 0.6390 - val_acc: 0.6878
Epoch 2/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.6313 - acc: 0.6395 - val_loss: 0.6036 - val_acc: 0.6878
Epoch 3/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.5995 - acc: 0.6670 - val_loss: 0.5718 - val_acc: 0.6878
Epoch 4/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.5686 - acc: 0.6796 - val_loss: 0.5414 - val_acc: 0.6878
Epoch 5/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.5389 - acc: 0.7156 - val_loss: 0.5086 - val_acc: 0.7539
Epoch 6/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.5118 - acc: 0.7677 - val_loss: 0.4721 - val_acc: 0.8044
Epoch 7/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4791 - acc: 0.8160 - val_loss

3087/3087 [==============================] - 9s 3ms/step - loss: 0.2806 - acc: 0.8944 - val_loss: 0.2763 - val_acc: 0.8873
Epoch 20/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2890 - acc: 0.8908 - val_loss: 0.2687 - val_acc: 0.9028
Epoch 21/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2857 - acc: 0.8899 - val_loss: 0.2692 - val_acc: 0.9016
Epoch 22/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2785 - acc: 0.8970 - val_loss: 0.2798 - val_acc: 0.8886
Epoch 23/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2828 - acc: 0.8947 - val_loss: 0.2750 - val_acc: 0.8964
Epoch 24/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2861 - acc: 0.8937 - val_loss: 0.2687 - val_acc: 0.9016
Epoch 25/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2831 - acc: 0.8915 - val_loss: 0.2863 - val_acc: 0.8912

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.0

3087/3087 [==============================] - 9s 3ms/step - loss: 0.2689 - acc: 0.8957 - val_loss: 0.2675 - val_acc: 0.9067
Epoch 138/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2678 - acc: 0.8970 - val_loss: 0.2676 - val_acc: 0.9067
Epoch 139/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2716 - acc: 0.8983 - val_loss: 0.2674 - val_acc: 0.9067
Epoch 140/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2732 - acc: 0.8947 - val_loss: 0.2676 - val_acc: 0.9067
Epoch 141/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2723 - acc: 0.8947 - val_loss: 0.2674 - val_acc: 0.9080
Epoch 142/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2728 - acc: 0.8960 - val_loss: 0.2675 - val_acc: 0.9080
Epoch 143/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2678 - acc: 0.8957 - val_loss: 0.2676 - val_acc: 0.9067
Epoch 144/200
3087/3087 [==============================

Epoch 197/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2721 - acc: 0.8973 - val_loss: 0.2676 - val_acc: 0.9067
Epoch 198/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2707 - acc: 0.8960 - val_loss: 0.2673 - val_acc: 0.9067
Epoch 199/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2686 - acc: 0.8954 - val_loss: 0.2675 - val_acc: 0.9067
Epoch 200/200
3087/3087 [==============================] - 9s 3ms/step - loss: 0.2657 - acc: 0.8954 - val_loss: 0.2674 - val_acc: 0.9067
Test accuracy: 0.9067357512953368
Train on 3087 samples, validate on 772 samples
Epoch 1/50
3087/3087 [==============================] - 10s 3ms/step - loss: 0.6208 - acc: 0.6479 - val_loss: 0.5635 - val_acc: 0.7176
Epoch 2/50
3087/3087 [==============================] - 9s 3ms/step - loss: 0.5327 - acc: 0.7234 - val_loss: 0.4282 - val_acc: 0.8199
Epoch 3/50
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4162 - acc: 0.8555 - val

In [3]:
#0.8152 acc

In [4]:
#0.9165 acc --> epochs 150, lr = 0.0005

In [ ]:
#0.899 acc --> epochs 200, lr = 0.001